In [1]:
# !pip install matplotlib
# !pip install seaborn
!pip install smogn

%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import smogn

  Using cached smogn-0.1.2-py3-none-any.whl (30 kB)


In [2]:
train_data=pd.read_csv('../Datasets/traindata.csv')
test_df=pd.read_csv('../Datasets/testdata.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/traindata.csv'

In [ ]:
train_data.columns
   

In [ ]:
train_data.describe()
zero_count=(train_data == 0).astype(int).sum(axis=0)
zero_count.plot(figsize=(14,5),kind='bar')


In [ ]:
train_data.info()


In [ ]:
def remove_dot_zeros_from_df(df):
    df=df.astype(object)
#     df=df.loc[:, (train_data != 0).any(axis=0)]  ### to remove columns containing all zeros
    df = train_data.loc[:, (train_data==0).mean() < .7]   #Drop columns with more than 70% zeros
    return df    

train_df=remove_dot_zeros_from_df(train_data)

In [ ]:
train_df['TARGET']=train_data['TARGET']


In [ ]:
train_df

In [ ]:
print('Non_zero values\n',train_df.astype(bool).sum(axis=0),'\n')
print('Zero\n',(train_df == 0).astype(int).sum(axis=0),'\n')


In [ ]:
# from IPython.display import IFrame
# wiki=IFrame('https://en.wikipedia.org/wiki/Pearson_correlation_coefficient',width=1000,height=400)
# display(wiki)
# features_df = train_data.loc[:, (train_data==0).mean() < .7]   #Drop columns with more than 70% zeros
# features_df['TARGET']=train_df['TARGET']
# features_df=features_df.astype(object)
# features_df


In [ ]:
plt.figure(figsize=(25,15)) 
sns.heatmap(train_df.corr(), annot=True,cmap="YlGnBu")
# sns.pairplot(features_df,height=1.3)

In [ ]:
X_wo_smogn=train_df.iloc[:,:-1]
y_wo_smogn=train_df.iloc[:,-1]



In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
regressor.fit(X_wo_smogn, y_wo_smogn)


In [ ]:
intersection_columns=X_wo_smogn.columns & test_df.columns
X_test_data=test_df[intersection_columns]

# print('features_Test_samples',X_test_data.shape)

y_pred = regressor.predict(X_test_data)
regressor.score(X_wo_smogn,y_wo_smogn)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# mean_absolute_percentage_error(y_wo_smogn,y_pred)

In [ ]:
# print(train_df['TARGET'].value_counts())
# # tf=train_df.cumsum()
# train_df['TARGET'].plot(kind='line')
# train_df[train_df['TARGET'].min(axis=0) > 0.03].loc[:, COL_HOME].min(axis=1)


### Synthetic Minority Over-Sampling Technique for Regression with Gaussian Noise(SMOGN)
A Python implementation of Synthetic Minority Over-Sampling Technique for Regression with Gaussian Noise (SMOGN). Conducts the Synthetic Minority Over-Sampling Technique for Regression (SMOTER) with traditional interpolation, as well as with the introduction of Gaussian Noise (SMOTER-GN). Selects between the two over-sampling techniques by the KNN distances underlying a given observation. If the distance is close enough, SMOTER is applied. If too far away, SMOTER-GN is applied. Useful for prediction problems where regression is applicable, but the values in the interest of predicting are rare or uncommon. This can also serve as a useful alternative to log transforming a skewed response variable, especially if generating synthetic data is also of interest.

In [ ]:
rg_mtrx_1 = [
    [4000000,  1, 0],  ## over-sample ("minority")
    [8000000, 0, 0],  ## under-sample ("majority")
    [1200000, 0, 0],  ## under-sample
    [2500000, 0, 0],  ## under-sample
]
train_df_smogn_1=smogn.smoter(train_df,y='TARGET',k=7,samp_method='balance',pert=0.04,rel_thres = 0.10,rel_method = 'manual',rel_ctrl_pts_rg = rg_mtrx_1)

In [ ]:
# print(list((train_df == 0).astype(int).sum(axis=0)))
# train_df_smogn_1=train_df_smogn_1.astype(object)
# print('Zero\n',(train_df == 0).astype(int).sum(axis=0),'\n')
# print('Zero\n',(train_df_smogn_1 == 0).astype(int).sum(axis=0),'\n')

zero_count=(train_df == 0).astype(int).sum(axis=0)
zero_count.plot(figsize=(14,5),kind='bar')
zero_cnt=(train_df_smogn_1 == 0).astype(int).sum(axis=0)
zero_cnt.plot(figsize=(14,5),kind='bar')


In [ ]:
# train_df_masked=train_df.mask(train_df==0).fillna(train_df.median())
# train_df_masked.head()
train_df_smogn_1.shape

In [ ]:
X=train_df_smogn_1.iloc[:,:-1]
y=train_df_smogn_1.iloc[:,-1]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
regressor.fit(X, y)
regressor.score(X,y)

In [ ]:
# Predicting a new result
y_pred = regressor.predict(X_test_data)
print('Predicted_score',y_pred.shape)



In [ ]:

# !pip install sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 30)
# creating a standard scaler
sc = StandardScaler()

# feeding independents sets into the standard scaler
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
regressor = RandomForestRegressor(n_estimators = 200)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
regressor.score(X_train,y_train)

In [ ]:
predicted_values=regressor.predict(X_test_data)
predicted_values

In [ ]:
test_df['Predicted_Amount']=predicted_values

In [ ]:
test_df.to_csv('../Datasets/Predicted_amount.csv')